Inspired by Deep Learning for Computer Vision with Python [Rosebrock]  
Chapter 17  
MiniVGGNet: CIFAR-10 with training monitor

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelBinarizer
from mini_vgg_net import mini_vgg_net

import os
import json
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
class TrainingMonitor(tf.keras.callbacks.BaseLogger):

    def __init__(self, fig_path, json_path, start_at=0):
        super(TrainingMonitor, self).__init__()
        self.fig_path = fig_path   # output path for the figure
        self.json_path = json_path # path to the JSON serialized file
        self.start_at = start_at   # the starting epoch

    def on_train_begin(self, logs={}):
        self.H = {} # initialize the history dictionary
        # load the training history if the JSON path already exists
        if self.json_path is not None:
            if os.path.exists(self.json_path):
                self.H = json.loads(open(self.json_path).read())
                # check to see if a starting epoch was supplied
                if self.start_at > 0:
                    # trim any entries that are past the starting epoch
                    for k in self.H.keys():
                        self.H[k] = self.H[k][:self.start_at]

    def on_epoch_end(self, epoch, logs={}):
        # loop over the logs and update the loss, accuracy, etc.
        for (k, v) in logs.items():
            l = self.H.get(k, [])
            l.append(float(v))
            self.H[k] = l

        # check to see if the training history should be serialized to file
        if self.json_path is not None:
            f = open(self.json_path, 'w')
            f.write(json.dumps(self.H))
            f.close()        # ensure at least two epochs have passed before plotting
        # (epoch starts at zero)
        if len(self.H['loss']) > 1:
            # plot the training loss and accuracy
            N = np.arange(0, len(self.H['loss']))
            plt.style.use('ggplot')
            plt.figure()
            plt.plot(N, self.H['loss'], label='train_loss')
            plt.plot(N, self.H['val_loss'], label='val_loss')
            plt.plot(N, self.H['accuracy'], label='train_acc')
            plt.plot(N, self.H['val_accuracy'], label='val_acc')
            plt.title('Training Loss and Accuracy [Epoch {}]'.format(len(self.H['loss'])))
            plt.xlabel('Epoch #')
            plt.ylabel('Loss/Accuracy')
            plt.legend()

            # save the figure
            plt.savefig(self.fig_path)
            plt.close()

In [ ]:
# load the CIFAR-10 dataset of 60,000 images of 10 classes
# 50,000 for training and 10,000 for testing
((trainX, trainY), (testX, testY)) = tf.keras.datasets.cifar10.load_data()

# scale data to the range of [0, 1]
trainX = trainX.astype('float32') / 255.0
testX = testX.astype('float32') / 255.0

# convert the labels from integers to vectors
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
model = mini_vgg_net()
model.summary()

# initialize the gradient descent optimizer
sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)

# compile the model
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
process_id = os.getpid()
print(f'process_id=')

# construct the set of callbacks
fig_path = f'{process_id}.png'
json_path = f'{process_id}.json'

# train the model
EPOCHS = 100
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs=EPOCHS, verbose=1,
              callbacks=[TrainingMonitor(fig_path, json_path)])